**Copyright 2020 The TensorFlow Authors.**

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Ejemplo de agrupación de pesos en Keras

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/model_optimization/guide/clustering/clustering_example"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/model_optimization/guide/clustering/clustering_example.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/model_optimization/guide/clustering/clustering_example.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/model_optimization/guide/clustering/clustering_example.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a>
</td>
</table>

## Descripción general

Le damos la bienvenida al ejemplo de principio a fin de *agrupación de pesos*, que forma parte del kit de herramientas de optimización de modelos de TensorFlow.

### Otras paginas

Para obtener una introducción a qué es la agrupación de pesos y determinar si debe usarla (incluido lo que se admite), consulte la [página de descripción general](https://www.tensorflow.org/model_optimization/guide/clustering).

Para encontrar rápidamente las API que necesita para su caso de uso (más allá de agrupar completamente un modelo con 16 clústeres), consulte la [guía completa](https://www.tensorflow.org/model_optimization/guide/clustering/clustering_comprehensive_guide).

### Contenido

En este tutorial podrá:

1. Entrenar un modelo `tf.keras` para el conjunto de datos MNIST desde cero.
2. Ajustar el modelo aplicando la API de agrupación de pesos y ver la precisión.
3. Crear modelos de TF y TFLite 6 veces más pequeños a partir de la agrupación.
4. Crear un modelo TFLite 8 veces más pequeño combinando la agrupación de pesos y la cuantización posentrenamiento.
5. Ver la persistencia de la precisión desde TF a TFLite.

## Preparación

Puede ejecutar este cuaderno Jupyter en su [virtualenv](https://www.tensorflow.org/install/pip?lang=python3#2.-create-a-virtual-environment-recommended) o [colab](https://colab.sandbox.google.com/) local. Para obtener detalles sobre la configuración de dependencias, consulte la [guía de instalación](https://www.tensorflow.org/model_optimization/guide/install). 

In [ ]:
! pip install -q tensorflow-model-optimization

In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import tempfile
import zipfile
import os

## Entrenar un modelo tf.keras para MNIST sin agrupación

In [ ]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images  = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(28, 28)),
    keras.layers.Reshape(target_shape=(28, 28, 1)),
    keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation=tf.nn.relu),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
    train_images,
    train_labels,
    validation_split=0.1,
    epochs=10
)

### Evaluar el modelo previsto y guardarlo para usarlo después

In [ ]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
print('Saving model to: ', keras_file)
tf.keras.models.save_model(model, keras_file, include_optimizer=False)

## Ajustar el modelo preentrenado con agrupación

Aplique la API `cluster_weights()` a todo un modelo preentrenado para demostrar su eficacia a la hora de reducir el tamaño del modelo después de aplicar la compresión y conservar una precisión aceptable. Para saber cuál es la mejor manera de equilibrar la precisión y la tasa de compresión para su caso de uso, consulte el ejemplo por capa en la [guía completa](https://www.tensorflow.org/model_optimization/guide/clustering/clustering_comprehensive_guide).


### Definir el modelo y aplicar la API de agrupación

Antes de pasar el modelo a la API de agrupación, asegúrese de que esté entrenado y muestre una precisión aceptable.

In [ ]:
import tensorflow_model_optimization as tfmot

cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 16,
  'cluster_centroids_init': CentroidInitialization.LINEAR
}

# Cluster a whole model
clustered_model = cluster_weights(model, **clustering_params)

# Use smaller learning rate for fine-tuning clustered model
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)

clustered_model.compile(
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer=opt,
  metrics=['accuracy'])

clustered_model.summary()

### Ajustar el modelo y evaluar la precisión con respecto a la línea de base

Ajuste el modelo con agrupación durante 1 época.

In [ ]:
# Fine-tune model
clustered_model.fit(
  train_images,
  train_labels,
  batch_size=500,
  epochs=1,
  validation_split=0.1)

Para este ejemplo, hay una pérdida mínima en la precisión de la prueba después de la agrupación, en comparación con la línea de base.

In [ ]:
_, clustered_model_accuracy = clustered_model.evaluate(
  test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Clustered test accuracy:', clustered_model_accuracy)

## Crear modelos **6 veces** más pequeños a partir de agrupaciones

Tanto `strip_clustering` como la aplicación de un algoritmo de compresión estándar (por ejemplo, mediante gzip) son necesarios para ver los beneficios de la compresión de la agrupación.

Primero, cree un modelo comprimible para TensorFlow. Aquí, `strip_clustering` elimina todas las variables (por ejemplo `tf.Variable` para almacenar los centroides y los índices de clúster) que la agrupación solo necesita durante el entrenamiento, que de otro modo aumentarían el tamaño del modelo durante la inferencia.

In [ ]:
final_model = tfmot.clustering.keras.strip_clustering(clustered_model)

_, clustered_keras_file = tempfile.mkstemp('.h5')
print('Saving clustered model to: ', clustered_keras_file)
tf.keras.models.save_model(final_model, clustered_keras_file, 
                           include_optimizer=False)

Luego, cree modelos comprimibles para TFLite. Puede convertir el modelo agrupado a un formato que se pueda ejecutar en su backend de destino. TensorFlow Lite es un ejemplo que se puede usar para implementar en dispositivos móviles.

In [ ]:
clustered_tflite_file = '/tmp/clustered_mnist.tflite'
converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
tflite_clustered_model = converter.convert()
with open(clustered_tflite_file, 'wb') as f:
  f.write(tflite_clustered_model)
print('Saved clustered TFLite model to:', clustered_tflite_file)

Defina una función ayudante para comprimir los modelos mediante gzip y medir el tamaño comprimido.

In [ ]:
def get_gzipped_model_size(file):
  # It returns the size of the gzipped model in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

Compare y vea que los modelos son **6 veces** más pequeños a aprtir de la agrupación

In [ ]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped clustered Keras model: %.2f bytes" % (get_gzipped_model_size(clustered_keras_file)))
print("Size of gzipped clustered TFlite model: %.2f bytes" % (get_gzipped_model_size(clustered_tflite_file)))

## Crear un modelo TFLite **8 veces** más pequeño combinando la agrupación de pesos y la cuantización posentrenamiento

Puede aplicar la cuantización posentrenamiento al modelo agrupado para obtener beneficios adicionales.

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

_, quantized_and_clustered_tflite_file = tempfile.mkstemp('.tflite')

with open(quantized_and_clustered_tflite_file, 'wb') as f:
  f.write(tflite_quant_model)

print('Saved quantized and clustered TFLite model to:', quantized_and_clustered_tflite_file)
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped clustered and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_clustered_tflite_file)))

## Ver la persistencia de la precisión desde TF a TFLite

Defina una función ayudante para evaluar el modelo de TFLite en el conjunto de datos de prueba.

In [ ]:
def eval_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

Evalúe el modelo, que ha sido agrupado y cuantizado, y luego vea que la precisión de TensorFlow persiste en el backend de TFLite.

In [ ]:
interpreter = tf.lite.Interpreter(model_content=tflite_quant_model)
interpreter.allocate_tensors()

test_accuracy = eval_model(interpreter)

print('Clustered and quantized TFLite test_accuracy:', test_accuracy)
print('Clustered TF test accuracy:', clustered_model_accuracy)

## Conclusión

En este tutorial, vio cómo crear modelos agrupados con la API del kit de herramientas de optimización de modelos de TensorFlow. Más específicamente, ha analizado un ejemplo de principio a fin para crear un modelo 8 veces más pequeño para MNIST con una diferencia de precisión mínima. Le recomendamos que pruebe esta nueva capacidad, que puede ser particularmente importante para la implementación en entornos con recursos limitados.
